In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Limpieza de la tabla de datos nutricionales por toma**

####**Descripción:**
Filtra y limpia la tabla de datos nutricionales asociados a cada toma, imputando datos faltantes, corrigiendo algunos z-scores dudosos y generando nuevas variables (`EdadDias`, `desnutricion`, `desnutricion_previa` y `desnutricion_siguiente`). Genera una nueva tabla de datos normalizada y limpia, con una fila por toma nutricional.
 
####**Requerimientos:**
-  Diccionario `datatypes_dictionaries/dtypes_var_toma.json`, que mapea cada columna de la tabla de datos nutricionales al *datatype* de Pandas más adecuado.
- Diccionario `datatypes_dictionaries/cat_tom.json`, que mapea cada columna categórica de la tabla de datos nutricionales a una lista (ordenada, si aplica) de sus categorías.
- Diccionario `datatypes_dictionaries/cols_ord.json` que incluye una lista de las variables categóricas ordinales de cada tabla de datos.
- Tabla de datos del beneficiario limpia `clean_data/beneficiario.parquet`.
- Función `impute_years_log` e  `impute_nans_range_log` de la librería `utils`.
- Software en línea [*Anthro Survey Analyser*](https://worldhealthorg.shinyapps.io/anthro/) de la Organización Mundial de la Salud, para el cálculo de z-scores.
 
####**Salidas:**
- Tablas de datos `auxiliary_data/z_scores_correction/tom_corregir_i.csv` (con `i`= 1, 2, 3) con las variables antropométricas de tomas nutricionales cuyos z-scores resultan dudosos.
- Tablas de datos `auxiliary_data/z_scores_correction/tom_corregir_i_zscore.csv` (con `i`= 1, 2, 3), similares a la anterior, pero con los z-scores recalculados por el software *Anthro Survey Analyser*.
- Tabla de datos `clean_data/tomas_nutricionales.parquet` con datos nutricionales de todas las tomas completamente corregidos y filtrados y datos faltantes asignados.
- Archivo `log_files/log_tomas_limpieza.txt` con el reporte de los datos nulos imputados al inicio.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
from utils import impute_years_log, impute_nans_range_log

### **Lectura del archivo**

In [ ]:
t0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/cat_tom.json', 'r') as file:
  cat_tom = json.load(file)
# Lee la tabla de datos nutricionales normalizada con los identificadores correctos
tom = pd.read_parquet('clean_data/tomas_normalizado_ids.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()
# Lee la tabla de datos de los beneficiarios limpia
ben = pd.read_parquet('clean_data/beneficiario.parquet')
# Lista de reportes de cambios realizados
log = list()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19275081 entries, 0 to 19275080
Data columns (total 27 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float32       
 9   Peso                            float32       
 10  Talla                           float32       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

### **Preliminares**

In [ ]:
# Duración promedio de un mes en días
dias_mes = 365.25 / 12
# Lista de variables categóricas
cols_cat = [col for col in tom.columns if col in cat_tom.keys()]
# Lista de variables categóricas ordinales
with open('datatypes_dictionaries/cols_ord.json', 'r') as file:
  cols_ord = json.load(file)['tom']
# Subconjuntos de variables útiles
cols_fecha = [col for col in tom.columns if col[:5] == 'Fecha']
cols_medidas = ['MedicionPerimetroBraquial', 'Peso', 'Talla']
cols_zscore = [col for col in tom.columns if col[:6] == 'ZScore']
cols_estado = [col for col in tom.columns if col[:6] == 'Estado']
# Lista de columnas que pueden cruzarse con la tabla de datos del beneficiario
cols_ben = ['IdBeneficiario', 'Sexo', 'FechaNacimiento']

In [ ]:
# Funciones de asignación del estado nutricional a partir de los z-scores

def estado_talla_edad(z_score_talla_edad):
  if -6 <= z_score_talla_edad < -2:
    return 'Retraso en talla'
  elif -2 <= z_score_talla_edad < -1:
    return 'Riesgo de baja talla'
  elif -1 <= z_score_talla_edad <= 3:
    return 'Talla adecuada para la edad'
  elif 3 < z_score_talla_edad <= 6:
    return 'Talla por encima de lo esperado'
  else:
    return np.nan

def estado_peso_edad(z_score_peso_edad):
  if -6 <= z_score_peso_edad < -3:
    return 'Desnutrición global severa'
  elif -3 <= z_score_peso_edad < -2:
    return 'Desnutrición global'
  elif -2 <= z_score_peso_edad < -1:
    return 'Riesgo de peso bajo para la edad'
  elif -1 <= z_score_peso_edad <= 1:
    return 'Peso adecuado para la edad'
  elif 1 < z_score_peso_edad <= 2:
    return 'Sobrepeso'
  elif 2 < z_score_peso_edad <= 5:
    return 'Obesidad'
  else:
    return np.nan

def estado_peso_talla(z_score_peso_talla):
  if -5 <= z_score_peso_talla < -3:
    return 'Desnutrición aguda severa'
  elif -3 <= z_score_peso_talla < -2:
    return 'Desnutrición aguda moderada'
  elif -2 <= z_score_peso_talla < -1:
    return 'Riesgo de desnutrición aguda'
  elif -1 <= z_score_peso_talla <= 1:
    return 'Peso adecuado para la talla'
  elif 1 < z_score_peso_talla <= 2:
    return 'Riesgo de sobrepeso'
  elif 2 < z_score_peso_talla <= 3:
    return 'Sobrepeso'
  elif 3 < z_score_peso_talla <= 5:
    return 'Obesidad'
  else:
    return np.nan

def estado_IMC(z_score_IMC):
  if -5 <= z_score_IMC < -2:
    return 'Delgadez'
  elif -2 <= z_score_IMC < -1:
    return 'Riesgo para la delgadez'
  elif -1 <= z_score_IMC <= 1:
    return 'Adecuado para la edad'
  elif 1 < z_score_IMC <= 2:
    return 'Riesgo de sobrepeso'
  elif 2 < z_score_IMC <= 3:
    return 'Sobrepeso'
  elif 3 < z_score_IMC <= 5:
    return 'Obesidad'
  else:
    return np.nan

estado_nutricional = {
  'TallaEdad': estado_talla_edad,
  'PesoEdad': estado_peso_edad,
  'PesoTalla': estado_peso_talla,
  'IMC': estado_IMC
  }

### **Borrado de espacios y exploración de rangos**

`CENTROS DE RECUPERACIÓN NUTRICIONAL PARA LA PRIMERA INFANCIA\xa0` $\rightarrow$ `CENTROS DE RECUPERACIÓN NUTRICIONAL PARA LA PRIMERA INFANCIA`.

In [ ]:
# Borra espacios al inicio y final de las categorías
col='Servicio'
tom[col] =  tom[col].astype('string').str.strip().astype('category')

In [ ]:
tom.agg([min, max]).transpose()

min                  max
IdToma                                            0             19275080
Registro                                          0              6007560
FechaValoracionNutricional      2017-01-01 00:00:00  2019-12-20 00:00:00
EdadMeses                                       -11                 1415
FechaMedicionPerimetroBraquial  1900-01-01 00:00:00  2027-01-05 00:00:00
MedicionPerimetroBraquial                         0                14444
Peso                                              0                 4350
Talla                                             0                600.2
ZScoreTallaEdad                              -27.34               256.55
ZScorePesoEdad                               -11.22              6270.56
ZScorePesoTalla                              -15.11              8673.54
ZScoreIMC                                    -15.91               178276
Flag                                              0                    7
FechaRegistroSaludNutricion     2016-11-14 00:00:00  2019-12-22 18:45:26
ControlesCrecimDesarrollo                         0                    4
IdBeneficiario                                   36             18973750
Id                                                3              5945451

### **Validación de categorías**

In [ ]:
print('Categorías no presentes en el diccionario de datos:')
for col in tom.columns:
  if col in cols_cat:
    cat_nulas = tom[~(tom[col].isin(cat_tom[col]))][col].unique()
    print(f"'{col}': {list(cat_nulas)}")

Categorías no presentes en el diccionario de datos:
'Vigencia': []
'Toma': []
'Servicio': []
'EstadoTallaEdad': ['_', nan, 'TALLA ADECUADA PARA LA EDAD', 'RETRASO EN TALLA PARA LA EDAD', 'TALLA POR ENCIMA DE LO ESPERADO']
'EstadoPesoEdad': [nan, '_', 'No Aplica', 'PESO ADECUADO PARA LA EDAD', 'BAJO PESO PARA LA EDAD', 'EXCESO DE PESO PARA LA EDAD']
'EstadoPesoTalla': [nan, '_']
'EstadoIMC': ['_', 'SIN EXCESO DE PESO', nan, 'EXCESO DE PESO']
'PresentaCarneVacunacion': [' ']
'AntecedentePremadurez': ['-', ' ', nan]
'Direccion': []


Como se observa, en las columnas de estado nutricional existen algunas categorías que no están en el diccionario de datos. Intentamos evidenciar cómo se distribuyen los z-scores correspondientes para estas categorías.

In [ ]:
for var in ['TallaEdad', 'PesoEdad', 'PesoTalla', 'IMC']:
  estado_cat_nulas = tom.loc[~tom[f'Estado{var}'].isin(cat_tom[f'Estado{var}']),
                             [f'Estado{var}', f'ZScore{var}']]
  n_valores = estado_cat_nulas[f'ZScore{var}'].nunique()
  print(f"\n'Estado{var}': Los registros en las categorías no descritas contienen " +
        f"{n_valores} valores distintos de z-score {var}.")
  print(f'Desviación estándar del z-score {var} en los registros con categorías no descritas:')
  display(estado_cat_nulas.groupby(f'Estado{var}').std().dropna())


'EstadoTallaEdad': Los registros en las categorías no descritas contienen 1 valores distintos de z-score TallaEdad.
Desviación estándar del z-score TallaEdad en los registros con categorías no descritas:


ZScoreTallaEdad
EstadoTallaEdad                                 
RETRASO EN TALLA PARA LA EDAD                0.0
TALLA ADECUADA PARA LA EDAD                  0.0
TALLA POR ENCIMA DE LO ESPERADO              0.0
_                                            0.0


'EstadoPesoEdad': Los registros en las categorías no descritas contienen 1546 valores distintos de z-score PesoEdad.
Desviación estándar del z-score PesoEdad en los registros con categorías no descritas:


ZScorePesoEdad
EstadoPesoEdad                             
BAJO PESO PARA LA EDAD             0.000000
EXCESO DE PESO PARA LA EDAD        0.000000
No Aplica                          2.573075
PESO ADECUADO PARA LA EDAD         0.000000
_                                  0.000000


'EstadoPesoTalla': Los registros en las categorías no descritas contienen 1 valores distintos de z-score PesoTalla.
Desviación estándar del z-score PesoTalla en los registros con categorías no descritas:


ZScorePesoTalla
EstadoPesoTalla                 
_                            0.0


'EstadoIMC': Los registros en las categorías no descritas contienen 11 valores distintos de z-score IMC.
Desviación estándar del z-score IMC en los registros con categorías no descritas:


ZScoreIMC
EstadoIMC                    
EXCESO DE PESO       0.000000
SIN EXCESO DE PESO   0.000000
_                    0.053735

Las categorías de estado nutricional en mayúscula sostenida (por ejemplo, `SIN EXCESO DE PESO`) siempre se asocian a valores de z-score nulos (0), que seguramente no fueron calculados en el momento de la toma de datos. Por lo anterior, no se pierde ninguna información al anular estas categorías extrañas.

Lo mismo es cierto para la categoría `No Aplica` y para los caracteres que se usaron para codificar los datos faltantes: espacio en blanco o guíones (`-` o `_`).

In [ ]:
for col in ['PresentaCarneVacunacion', 'AntecedentePremadurez']:
  print(f"\nEn '{col}': Composición de registros.")
  display(tom[~tom[col].isin(cat_tom[col])][col].value_counts().to_frame())


En 'PresentaCarneVacunacion': Composición de registros.


PresentaCarneVacunacion
                       475
S                        0
N                        0


En 'AntecedentePremadurez': Composición de registros.


AntecedentePremadurez
-               18869674
                   18357
S                      0
N                      0

In [ ]:
nan_inicio = tom.isna()
# Forzar categorías y ordenar las categorías
for col in cols_cat:
  categorias = cat_tom[col]
  es_ordinal = col in cols_ord
  tom[col] = tom[col].cat.set_categories(categorias, ordered = es_ordinal)
nulos = tom.isna()
nan_nuevos = (nulos & ~nan_inicio).sum()
print(f'Nuevos NaNs imputados (categorías no válidas):')
nan_nuevos[nan_nuevos > 0]

Nuevos NaNs imputados (categorías no válidas):


EstadoTallaEdad                4856
EstadoPesoEdad               980160
EstadoPesoTalla                4868
EstadoIMC                      5243
PresentaCarneVacunacion         475
AntecedentePremadurez      18888031
dtype: int64

### **Imputación de valores nulos**

In [ ]:
# Imputar nans
# Para fechas
nan_fechas = {pd.to_datetime('1900 - 1 - 1'),
             pd.to_datetime('1970 - 1 - 1')}
# Para medidas de peso, talla y perímetro braquial
nan_medidas = [0]
for col in cols_fecha:
  nan_inicio = tom[col].isna().sum()
  for fecha in nan_fechas:
    tom[col] = tom[col].replace(fecha, np.NaN)
  nan_nuevos = tom[col].isna().sum() - nan_inicio
  print(f"En '{col}', {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

for col in cols_medidas:
  nan_inicio = tom[col].isna().sum()
  for medida in nan_medidas:
    tom[col] = tom[col].replace(medida, np.NaN)
  nan_nuevos = tom[col].isna().sum() - nan_inicio
  print(f"En '{col}', {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

En 'FechaValoracionNutricional', 0 NaNs imputados: 132.24 s.
En 'FechaMedicionPerimetroBraquial', 11552290 NaNs imputados: 132.73 s.
En 'FechaRegistroSaludNutricion', 0 NaNs imputados: 133.16 s.
En 'MedicionPerimetroBraquial', 11536099 NaNs imputados: 133.37 s.
En 'Peso', 73 NaNs imputados: 133.52 s.
En 'Talla', 216 NaNs imputados: 133.68 s.


In [ ]:
# Vigencia de los datos
fecha_min = pd.to_datetime('2017 - 1 - 1')
fecha_max = pd.to_datetime('2019 - 12 - 31')

rango = {
  'FechaValoracionNutricional': [fecha_min, fecha_max],
  'EdadMeses': [0, 8 * 12],
  'FechaMedicionPerimetroBraquial': [fecha_min, fecha_max],
  'MedicionPerimetroBraquial': [5, 40],
  'Peso': [0.5, 100],
  'Talla': [20, 200],
  'ZScoreTallaEdad': [-6, 6],
  'ZScorePesoEdad': [-6, 5],
  'ZScorePesoTalla': [-5, 5],
  'ZScoreIMC': [-5, 5],
  }
# Imputar años de vigencia en las fechas de medición que caen fuera de los rangos
fuera_de_rango = (tom['FechaMedicionPerimetroBraquial'] < fecha_min) | \
  (tom['FechaMedicionPerimetroBraquial'] > fecha_max)

tom.loc[fuera_de_rango, 'FechaMedicionPerimetroBraquial'] = \
  impute_years_log(data=tom.loc[fuera_de_rango], col='FechaMedicionPerimetroBraquial', 
                   col_id='Registro', col_year='Vigencia', log=log)
nan_nuevos = fuera_de_rango.sum()
print(f"'{col}': {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

# Reemplazar los valores por fuera de sus rangos naturales por NaNs
for col in rango.keys():
  fuera_de_rango = (tom[col] < rango[col][0]) | (tom[col] > rango[col][1])
  tom.loc[fuera_de_rango, col] = \
    impute_nans_range_log(data=tom.loc[fuera_de_rango], col=col, 
                          col_id='Registro', range=rango[col], log=log)
  nan_nuevos = fuera_de_rango.sum()
  print(f"'{col}': {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

'Talla': 10 NaNs imputados: 134.15 s.
'FechaValoracionNutricional': 0 NaNs imputados: 134.45 s.
'EdadMeses': 3050 NaNs imputados: 134.92 s.
'FechaMedicionPerimetroBraquial': 0 NaNs imputados: 135.33 s.
'MedicionPerimetroBraquial': 953 NaNs imputados: 135.61 s.
'Peso': 35 NaNs imputados: 135.79 s.
'Talla': 9 NaNs imputados: 135.96 s.
'ZScoreTallaEdad': 60694 NaNs imputados: 141.68 s.
'ZScorePesoEdad': 13583 NaNs imputados: 143.11 s.
'ZScorePesoTalla': 45156 NaNs imputados: 147.40 s.
'ZScoreIMC': 61448 NaNs imputados: 153.19 s.


### **Asignación de `Sexo` y `FechaNacimiento` a partir de la base de datos de los beneficiario**

In [ ]:
tom = tom.merge(ben[cols_ben], on='IdBeneficiario', how='left')
ben['Sexo'].value_counts(normalize=True)

M    0.50532
F    0.49468
Name: Sexo, dtype: float64

In [ ]:
# Cantidad de datos faltantes (NaNs) por columna
nulos = tom.isna().sum()
nulos[nulos > 0]

FechaValoracionNutricional          100229
EdadMeses                             3053
FechaMedicionPerimetroBraquial    14590933
MedicionPerimetroBraquial         14465136
Peso                                   108
Talla                                  225
ZScoreTallaEdad                      61603
ZScorePesoEdad                       14492
ZScorePesoTalla                      46065
ZScoreIMC                            62357
EstadoTallaEdad                      10772
EstadoPesoEdad                     1852168
EstadoPesoTalla                     883001
EstadoIMC                            10783
Flag                                  5777
FechaRegistroSaludNutricion        6232755
PresentaCarneVacunacion                475
ControlesCrecimDesarrollo           288718
AntecedentePremadurez             19078044
Id                                10066283
Sexo                                 14804
FechaNacimiento                      10790
dtype: int64

### **Cálculo de la edad en días**
Suponemos que el cálculo de los z-scores será más preciso si se tiene la edad exacta de cada beneficiario para cada toma, por lo que se toma en días.

In [ ]:
tom['EdadDias'] = (tom['FechaValoracionNutricional'] - tom['FechaNacimiento']).dt.days
tom[['EdadMeses', 'EdadDias']].isna().sum()

EdadMeses      3053
EdadDias     107418
dtype: int64

Desafortunadamente, existen muchas fechas faltantes en la tabla de datos lo que impide el cálculo de la edad en días.

En los casos en que la `EdadDias` no pueda calcularse a partir de la `FechaNacimiento` y la `FechaValoracionNutricional`, se calculará de la `EdadMeses` asumiendo que el beneficiario se encuentra a medio mes de cumplir un nuevo mes.

In [ ]:
tom['EdadDias'] = tom['EdadDias'].fillna((tom['EdadMeses'] + 0.5) * dias_mes).round()
tom['EdadMeses'] = tom['EdadMeses'].fillna(tom['EdadDias'] / dias_mes).round(2)
tom[['EdadMeses', 'EdadDias']].isna().sum()

EdadMeses    260
EdadDias     260
dtype: int64

### **Filtrado de beneficiarios por edades**
Aunque se admite la posibilidad de que haya algunos errores en el reporte de la fecha de nacimiento que causen que los niños aparezcan como nacidos *después* de su valoración nutricional (`FechaNacimiento` > `FechaValoracionNutricional`, y por ende, `EdadDias` negativa), se descartan los datos donde este error sea mayor a medio mes (`EdadDias` < -15 días $\rightarrow$ 0 días).

Se eliminan también los datos de los beneficiarios que no tienen ninguna toma en el rango de edades objetivo (`EdadDias` > 1826 días = 5 años), pero se preservan aquellos que presentan datos antes y después de cumplir los 5 años de edad.

In [ ]:
# Beneficiarios por fuera del rango de edades
edad_min = -dias_mes / 2
edad_max = dias_mes * 60
# Beneficiarios que presentaron una edad negativa (por debajo de medio mes negativo)
con_edad_negativa = set(tom.loc[tom['EdadDias'] < edad_min, 'IdBeneficiario'])
# Beneficiarios que nunca estuvieron en la población de interés (edad <= 5 años).
min_edad = tom.groupby('IdBeneficiario')['EdadDias'].min().reset_index()
mayores = set(min_edad.loc[min_edad['EdadDias'] > edad_max, 'IdBeneficiario'])
# Elimina los beneficiarios descartados
IdB_no_validos = con_edad_negativa.union(mayores)
tom = tom[~tom['IdBeneficiario'].isin(IdB_no_validos)]
print(f'{len(IdB_no_validos)} beneficiarios por fuera del rango de edad:')
print(f'- {len(con_edad_negativa)} beneficiarios con edad negativa (fecha de nacimiento errónea).')
print(f'- {len(mayores)} beneficiarios mayores de 5 años (fuera de la población objetivo).')
# Convierte en 0 las edades negativas
for col in ['EdadMeses', 'EdadDias']:
  tom[col] = tom[col].apply(lambda x: max(x, 0))

85897 beneficiarios por fuera del rango de edad:
- 49 beneficiarios con edad negativa (fecha de nacimiento errónea).
- 85848 beneficiarios mayores de 5 años (fuera de la población objetivo).


In [ ]:
tom.agg([min, max]).transpose()

min                          max
IdToma                                                   0                     19275080
Registro                                                 0                      6007560
Vigencia                                              2017                         2019
Toma                                                     1                          12N
FechaValoracionNutricional             2017-01-01 00:00:00          2019-12-20 00:00:00
EdadMeses                                                0                           96
FechaMedicionPerimetroBraquial         2017-01-01 00:00:00          2019-12-20 00:00:00
MedicionPerimetroBraquial                                5                           40
Peso                                                   0.5                          100
Talla                                                   30                          186
ZScoreTallaEdad                                         -6                            6
ZScorePesoEdad                                          -6                            5
ZScorePesoTalla                                         -5                            5
ZScoreIMC                                               -5                            5
EstadoTallaEdad                           Retraso en talla  Talla adecuada para la edad
EstadoPesoEdad                  Desnutrición global severa                     Obesidad
EstadoPesoTalla                  Desnutrición aguda severa                     Obesidad
EstadoIMC                                         Delgadez                     Obesidad
Flag                                                     0                            7
FechaRegistroSaludNutricion            2016-11-14 00:00:00          2019-12-22 18:45:26
ControlesCrecimDesarrollo                                0                            4
IdBeneficiario                                          36                     18973750
Id                                                       3                      5945451
FechaNacimiento                        1978-08-28 00:00:00          2019-12-17 00:00:00
EdadDias                                                 0                         2860

### **Extracción y corrección de z-scores dudosos**
Se observa que en algunos registros los z-scores parecen no haber sido calculados mediante el software adecuado, sino estimados mediante algún otro método, generando valores enteros o simplemente 0.

Extraemos los registros que presentan estos valores sospechosos para recalcularlos.

In [ ]:
# Extrae los z-scores enteros (z-score mod 1 = 0) o nulos (NaNs)
con_z_score_dudoso = tom[cols_zscore].mod(1, fill_value=0) == 0
# Cuenta el número de z-scores dudosos por registro
z_scores_dudosos = con_z_score_dudoso.apply(sum, axis='columns')
print('Registros con z-scores enteros o nulos:')
z_scores_dudosos.value_counts()

Registros con z-scores enteros o nulos:


0    17351279
1      870736
2      717001
4      107584
3       37496
dtype: int64

Utilizamos el software [*Anthro Survey Analyser*](https://worldhealthorg.shinyapps.io/anthro/) de la Organización Mundial de la Salud para recalcular los z-scores.

Como este software solo admite registros que tengan un sexo definido, a las tomas que no presenten `Sexo` se les asignará la moda de la tabla de datos (`M`), esperando que las afectación a los resultados sean insignificantes comparadas con la ganancia de información.

El software también elimina automáticamente los registros con edad a partir de los 5 años cumplidos, por lo que a las tomas de beneficiarios mayores a 5 años, se les asignará la edad máxima (1826 días), con el propósito de generar los z-scores que no dependen de la edad (`ZScorePesoTalla` y `ZScoreIMC`) descartando luego los que sí dependen de esta (`ZScoreTallaEdad` y `ZScorePesoEdad`).

In [ ]:
# Tomas nutricionales con más de un valor de z-score dudoso
tom_corregir = tom.loc[z_scores_dudosos > 1, ['IdToma', 'Sexo', 'EdadDias', 'Peso', 'Talla']]
# Tomas por corregir con edad admisible por el software Anthro Survey Analyser ('EdadDias' <= 1826)
con_edad_valida = tom_corregir.loc[tom_corregir['EdadDias'] <= 1826, 'IdToma']
print(f'{len(con_edad_valida)} de {len(tom_corregir)} tomas por corregir, con edad en el rango adecuado.')
# Limita la edad al máximo permitido por el software (1826 días)
tom_corregir['EdadDias'] = tom_corregir['EdadDias'].apply(lambda x: min(x, 1826))
tom_corregir[['Peso', 'Talla']] = tom_corregir[['Peso', 'Talla']].round(2)
# Imputa el sexo mayoritario a los registros sin 'Sexo'
tom_corregir['Sexo'] = tom_corregir['Sexo'].fillna(tom_corregir['Sexo'].mode()[0])
# Divide la tabla en tres fragmentos similares y los guarda en un archivo csv
fragmento = np.array_split(tom_corregir, 3)
for i in range(3):
  fragmento[i].to_csv(f'auxiliary_data/z_scores_correction/tom_corregir_{i + 1}.csv')
tom_corregir

186336 de 862081 tomas por corregir, con edad en el rango adecuado.


IdToma Sexo  EdadDias       Peso  Talla
53              53    M    1826.0  17.000000  109.0
54              54    M    1826.0  18.600000  111.0
55              55    M    1826.0  18.910000  113.0
56              56    M    1826.0  19.600000  116.0
57              57    M    1826.0  19.299999  116.0
...            ...  ...       ...        ...    ...
19275076  19275076    M     228.0   8.000000   69.0
19275077  19275077    M     228.0   9.000000   72.0
19275078  19275078    M     289.0   9.000000   74.0
19275079  19275079    M     228.0   7.000000   68.0
19275080  19275080    M     380.0   8.000000   70.0

[862081 rows x 5 columns]

Las tablas de datos generadas en formato `csv` deben descargarse e introducirse manualmente en el [*Anthro Survey Analyser*](https://worldhealthorg.shinyapps.io/anthro/), indicando el significado de cada columna. El software calcula los z-scores correspondientes y permite descargar archivos `csv` con un nombre similar al archivo original y el prefijo `_zscore`. El proceso completo tarda unos 5 a 10 minutos.

Los archivos con los z-scores deben luego guardarse en el directorio `auxiliary_data/z_score_correction/`.

In [ ]:
#input("Presiona 'Enter' al terminar de cargar los archivos en el directorio " +
#      "'auxiliary_data/z_score_correction/': ")

In [ ]:
# Une las tablas de datos con los z-scores
fragmentos = list()
for i in range(3):
  fragmento = pd.read_csv(f'auxiliary_data/z_scores_correction/tom_corregir_{i + 1}_zscore.csv')
  fragmentos.append(fragmento)
correccion = pd.concat(fragmentos)[['IdToma', 'zlen', 'zwei', 'zbmi', 'zwfl']]
correccion

IdToma  zlen  zwei  zbmi  zwfl
0             53 -0.21 -0.56 -0.70 -0.77
1             54  0.22  0.11 -0.07 -0.17
2             55  0.66  0.23 -0.30 -0.41
3             56  1.30  0.49 -0.49 -0.64
4             57  1.30  0.38 -0.67 -0.81
...          ...   ...   ...   ...   ...
287355  19275076 -0.40 -0.51 -0.36 -0.30
287356  19275077  0.97  0.57  0.04  0.18
287357  19275078  0.61 -0.03 -0.50 -0.39
287358  19275079 -0.86 -1.72 -1.67 -1.61
287359  19275080 -2.63 -1.81 -0.31 -0.63

[862081 rows x 5 columns]

In [ ]:
# Imputa NaNs en los z-scores que involucran la edad, para las tomas con una edad mayor al límite
correccion.loc[~correccion['IdToma'].isin(con_edad_valida), ['zlen', 'zwei']] = np.NaN
correccion

IdToma  zlen  zwei  zbmi  zwfl
0             53   NaN   NaN -0.70 -0.77
1             54   NaN   NaN -0.07 -0.17
2             55   NaN   NaN -0.30 -0.41
3             56   NaN   NaN -0.49 -0.64
4             57   NaN   NaN -0.67 -0.81
...          ...   ...   ...   ...   ...
287355  19275076 -0.40 -0.51 -0.36 -0.30
287356  19275077  0.97  0.57  0.04  0.18
287357  19275078  0.61 -0.03 -0.50 -0.39
287358  19275079 -0.86 -1.72 -1.67 -1.61
287359  19275080 -2.63 -1.81 -0.31 -0.63

[862081 rows x 5 columns]

In [ ]:
nombre = {
  'ZScoreTallaEdad': 'zlen',
  'ZScorePesoEdad': 'zwei',
  'ZScorePesoTalla': 'zwfl',
  'ZScoreIMC': 'zbmi'
  }
tom = tom.merge(correccion, on='IdToma', how='left')
for col in cols_zscore:
  col_nueva = tom[nombre[col]].fillna(tom[col])
  tom[col] = col_nueva
  tom = tom.drop(columns=[nombre[col]])
# Cantidad de datos faltantes (NaNs) por columna de z-score
tom[cols_zscore].isna().sum()

ZScoreTallaEdad    32151
ZScorePesoEdad       810
ZScorePesoTalla     5009
ZScoreIMC          13668
dtype: int64

### **Imputación de valores nulos**

In [ ]:
# Reemplazar los valores por fuera de sus rangos naturales por NaNs
for col in cols_zscore:
  fuera_de_rango = (tom[col] < rango[col][0]) | (tom[col] > rango[col][1])
  tom.loc[fuera_de_rango, col] = \
    impute_nans_range_log(data=tom.loc[fuera_de_rango], col=col, 
                          col_id='Registro', range=rango[col], log=log)
  nan_nuevos = fuera_de_rango.sum()
  print(f"'{col}': {nan_nuevos} NaNs imputados: {time.time()-t0:.2f} s.")

'ZScoreTallaEdad': 26458 NaNs imputados: 363.94 s.
'ZScorePesoEdad': 13162 NaNs imputados: 365.38 s.
'ZScorePesoTalla': 42896 NaNs imputados: 369.49 s.
'ZScoreIMC': 48812 NaNs imputados: 374.12 s.


In [ ]:
# Cantidad de datos faltantes (NaNs) por columna de z-score
tom[cols_zscore].isna().sum()

ZScoreTallaEdad    58609
ZScorePesoEdad     13972
ZScorePesoTalla    47905
ZScoreIMC          62480
dtype: int64

### **Corrección de estado nutricional**

In [ ]:
# Recalcula el estado nutricional con base en los z-scores corregidos
for var in ['TallaEdad', 'PesoEdad', 'PesoTalla', 'IMC']:
  inicio = tom[f'Estado{var}']
  tom[f'Estado{var}'] = tom[f'ZScore{var}'].apply(estado_nutricional[var])
  n_cambios = (tom[f'Estado{var}'].ne(inicio) & inicio.notnull()).sum()
  print(f"En 'Estado{var}', {n_cambios} correcciones: {time.time() - t0:.2f} s.")

En 'EstadoTallaEdad', 170129 correcciones: 383.35 s.
En 'EstadoPesoEdad', 51682 correcciones: 394.88 s.
En 'EstadoPesoTalla', 91421 correcciones: 406.74 s.
En 'EstadoIMC', 337320 correcciones: 416.95 s.


In [ ]:
# Asigna los tipos de datos adecuados
tom = tom.astype(dtypes_vars_toma)
tom['EdadDias'] = tom['EdadDias'].astype('Int32')
# Reordena las categorías
for col in cols_cat:
  categorias = cat_tom[col]
  es_ordinal = col in cols_ord
  tom[col] = tom[col].cat.set_categories(categorias, ordered = es_ordinal)
# Ordena los registros por 'IdBeneficiario', por 'EdadDias' y por 'Toma' de modo que 
# para cada beneficiario los registros se ordenan cronológicamente
# Los registros con edad nula quedarán al final de cada beneficiario ordenados por 'Toma'
tom = tom.sort_values(['IdBeneficiario', 'EdadDias', 'Toma']).reset_index(drop=True)

In [ ]:
# Cantidad de datos faltantes (NaNs) por columna  de estado nutricional
nulos = tom[cols_estado].isna().sum()
nulos[nulos > 0]

EstadoTallaEdad    58609
EstadoPesoEdad     13972
EstadoPesoTalla    47905
EstadoIMC          62480
dtype: int64

### **Cálculo de desnutrición actual. anterior y siguiente**

Se calcula la presencia o ausencia de *desnutrición aguda* para cada toma nutricional (definida `ZScorePesoTalla` $<-2$) como la variable objetivo.

También se calcula el número de veces que el beneficiario ha presentado desnutrición en alguna toma anterior (`desnutricion_previa`) y si la presentará en la toma inmediatamente siguiente (`desnutricion_siguiente`).

In [ ]:
max_tomas = tom.groupby('IdBeneficiario')['IdBeneficiario'].count().max()
max_tomas

72

In [ ]:
# Desnutrición en la toma actual
tom['desnutricion'] = (tom['ZScorePesoTalla'] < -2)
# Desnutrición en la toma inmediatamente anterior
tom['desnutricion_previa0'] = tom['desnutricion'].shift(1) & \
  (tom['IdBeneficiario'] == tom['IdBeneficiario'].shift(1))
# Desnutrición en cada una de las tomas anteriores (máximo 72)
for i in range(1, max_tomas):
  tom[f'desnutricion_previa{i}'] = tom[f'desnutricion_previa{i - 1}'].shift(1) & \
    (tom['IdBeneficiario'] == tom['IdBeneficiario'].shift(1))
cols_desnutricion_previa = [f'desnutricion_previa{i}' for i in range(max_tomas)]
tom[cols_desnutricion_previa] = tom[cols_desnutricion_previa].fillna(False)
tom['desnutricion_previa'] = 0
# Desnutrición en alguna de las tomas anteriores
for i in range(0, max_tomas):
  tom['desnutricion_previa'] = tom['desnutricion_previa'] + (tom[f'desnutricion_previa{i}']).astype('int')
  tom = tom.drop(columns=[f'desnutricion_previa{i}'])
print(f"'desnutricion_previa': {(tom['desnutricion_previa'] > 0).sum()} " + 
      f"tomas con desnutrición previa asignadas: {time.time() - t0:.2f} s.")
tom['desnutricion_previa'].value_counts()[range(13)]

'desnutricion_previa': 853853 tomas con desnutrición previa asignadas: 503.66 s.


0     18230243
1       639653
2       138914
3        43937
4        16403
5         7113
6         3608
7         1938
8         1043
9          544
10         307
11         144
12          85
13          36
14          32
17          18
15          17
51          16
18           7
16           6
Name: desnutricion_previa, dtype: int64

In [ ]:
# Desnutrición en la toma inmediatamente siguiente
tom['desnutricion_siguiente'] = (tom['ZScorePesoTalla'].shift(-1) < -2) & \
  (tom['IdBeneficiario'].shift(-1) == tom['IdBeneficiario'])
tom['desnutricion_siguiente'] = tom['desnutricion_siguiente'].fillna(False)
print(f"'desnutricion_siguiente': {tom['desnutricion_siguiente'].sum()} " + 
      f"tomas con desnutrición siguiente asignadas: {time.time() - t0:.2f} s.")

'desnutricion_siguiente': 172582 tomas con desnutrición siguiente asignadas: 504.04 s.


In [ ]:
# Cantidad de datos faltantes (NaNs) por columna
nulos = tom.isna().sum()
nulos[nulos > 0]

FechaValoracionNutricional          100133
EdadMeses                              249
FechaMedicionPerimetroBraquial    14413643
MedicionPerimetroBraquial         14288177
Peso                                    96
Talla                                  220
ZScoreTallaEdad                      58609
ZScorePesoEdad                       13972
ZScorePesoTalla                      47905
ZScoreIMC                            62480
EstadoTallaEdad                      58609
EstadoPesoEdad                       13972
EstadoPesoTalla                      47905
EstadoIMC                            62480
Flag                                  5100
FechaRegistroSaludNutricion        6102786
PresentaCarneVacunacion                466
ControlesCrecimDesarrollo           287201
AntecedentePremadurez             18887085
Id                                 9946684
Sexo                                 14778
FechaNacimiento                      10632
EdadDias                               249
dtype: int6

### **Guardado**

In [ ]:
# Guarda la tabla de datos nutricionales limpia
tom.to_parquet('clean_data/tomas_nutricionales.parquet')
# Escribe un archivo con los cambios generados
with open('log_files/log_tomas_limpieza.txt', 'w') as log_file:
  log_file.write('Integracion_beneficiario:\n')
  for line in log:
    log_file.write('\t' + line + '\n')
print(f'Archivos guardados: {time.time() - t0:.2f} s.')

Archivos guardados: 535.62 s.
